In [ ]:
# Original Author: Michael(Mike) Hann
'''
Purpose: To ingest, interpolate, filter/correct, and display Emlid Reach M2 
GNSS and Ouster Lidar OS2 data using an Extended Kalman Filter
'''

'''
 To Do List/Workflow:
1. Split up GNSS and IMU data frames (speed things WAY up)
    a. Manipulate GNSS df
    b. Merge dfs back together
2. Add velocity vectors (x, y, z) to gnss_df from GNSS values
3. Trim all values under a certain velocity (make sure truck is moving)
4. Obtain intial heading value from GNSS positions (for yaw to manipulate)
2. Implement Complementary filter (properly this time)
3. Develop forward motion algorithm from Comp filter
x. Define 'optimized' values from filter/normal data
 
99. develop the UKF and pass the data through

'''

# Import Required Packages

import numpy as np
import pandas as pd
from os import chdir, getcwd
from time import perf_counter
import matplotlib.pyplot as plt


In [73]:
# INGEST FILE

# Set and Get directory
chdir(r'C:\Users\mikeh\OneDrive\Documents\GitHub\ouster_localization')
dir = getcwd()
print(f'\n\ndirectory: {dir}\n\n')

# Set the input file (full or small)
infile = 'C2_IMU.txt'
#infile = 'less_data.txt'

# Start performance counter
t1 = perf_counter()

print('\treading file...\n')
# Import the comma delimited .txt file as a pandas dataframe
df = pd.read_csv(f'{dir}\\{infile}', delimiter=',')

print('\tediting data types...\n')
# Extract only the numbers from the 'Time' column using a reg ex, convert to long integer
df['Time'] = df['Time'].str.extract('(\d+)').astype('float')

# Convert Time into seconds from onset
t0 = df.Time[0]
df.Time = (df.Time-t0)/10**9

# Set any future interpolated GNSS values to status 99
# Set any future interpolated standard deviations as to value of previous GNSS
df['GPS_Status'] = df['GPS_Status'].replace(' None', '99')
df.loc[:, 'SDn':'SDu'] = df.loc[:, 'SDn':'SDu'].replace(' None', pd.NA)
df.loc[:, 'SDn':'SDu'] = df.loc[:, 'SDn':'SDu'].astype('string').interpolate(method='ffill').astype(float)

# Set all values GNSS and IMU 'None' values to null, convert all object data types to presumed data types
df = df.replace(' None', pd.NA).convert_dtypes(infer_objects=True)
df.loc[:, 'GPS_Long':'GPS_Alt'] = df.loc[:, 'GPS_Long':'GPS_Alt'].astype('float')

print(df.info())

# End performance counter
t2 = perf_counter()
print(f'time: {t2-t1}\n')



directory: C:\Users\mikeh\OneDrive\Documents\GitHub\ouster_localization


	reading file...

	editing data types...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31376 entries, 0 to 31375
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Time            31376 non-null  Float64
 1   GPS_Long        1495 non-null   float64
 2   GPS_Lat         1495 non-null   float64
 3   GPS_Alt         1495 non-null   float64
 4   SDn             31376 non-null  Float64
 5   SDe             31376 non-null  Float64
 6   SDu             31376 non-null  Float64
 7   GPS_Status      31376 non-null  string 
 8   IMU_AngVelX     29881 non-null  string 
 9   IMU_AngVelY     29881 non-null  string 
 10  IMU_AngVelZ     29881 non-null  string 
 11  IMU_LinearAccX  29881 non-null  string 
 12  IMU_LinearAccY  29881 non-null  string 
 13  IMU_LinearAccZ  29881 non-null  string 
dtypes: Float64(4), float64(3), string(7)
memory usag

In [74]:
# SPLIT UP GNSS AND IMU DATA

t1 = perf_counter()

# Split into GNSS components
gnss_df = df.loc[:, 'Time':'SDu']

'''
imu_df = df.loc[:, 'Time','IMU_AngVelX':'IMU_LinearAccZ']
print(imu_df.head())'''


# ADD VELOCITY TO GNSS DATA

gnss_df['VelX'] = list(np.zeros(len(gnss_df.GPS_Long)))
gnss_df['VelY'] = list(np.zeros(len(gnss_df.GPS_Long)))
gnss_df['VelZ'] = list(np.zeros(len(gnss_df.GPS_Long)))

print(gnss_df.head())

print(f'\tadding velocities\n')

for i in range(2, len(gnss_df.GPS_Long)):
    
    if not gnss_df.GPS_Long[i]:
        break
    
    # Get change in time 
    dt = gnss_df.Time[i] - gnss_df.Time[i-1]

    # Return change
    gnss_df.VelX[i] = (gnss_df.GPS_Long[i] - gnss_df.GPS_Long[i-1]) / dt
    gnss_df.VelY[i] = (gnss_df.GPS_Lat[i] - gnss_df.GPS_Lat[i-1]) / dt
    gnss_df.VelZ[i] = (gnss_df.GPS_Alt[i] - gnss_df.GPS_Alt[i-1]) / dt
    
print(gnss_df.head())


t2 = perf_counter()
print(f'time: {t2-t1}')


       Time  GPS_Long    GPS_Lat  GPS_Alt       SDn       SDe       SDu  VelX  \
0       0.0 -65.07666  44.955054    8.595  0.000256  0.000256  0.004096   0.0   
1  0.013138       NaN        NaN      NaN  0.000256  0.000256  0.004096   0.0   
2  0.013267       NaN        NaN      NaN  0.000256  0.000256  0.004096   0.0   
3  0.015559       NaN        NaN      NaN  0.000256  0.000256  0.004096   0.0   
4  0.050943       NaN        NaN      NaN  0.000256  0.000256  0.004096   0.0   

   VelY  VelZ  
0   0.0   0.0  
1   0.0   0.0  
2   0.0   0.0  
3   0.0   0.0  
4   0.0   0.0  
	adding velocities



C:\Users\mikeh\AppData\Local\Temp/ipykernel_20508/1416534321.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnss_df.VelX[i] = (gnss_df.GPS_Long[i] - gnss_df.GPS_Long[i-1]) / dt
C:\Users\mikeh\AppData\Local\Temp/ipykernel_20508/1416534321.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnss_df.VelY[i] = (gnss_df.GPS_Lat[i] - gnss_df.GPS_Lat[i-1]) / dt
C:\Users\mikeh\AppData\Local\Temp/ipykernel_20508/1416534321.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

       Time  GPS_Long    GPS_Lat  GPS_Alt       SDn       SDe       SDu  VelX  \
0       0.0 -65.07666  44.955054    8.595  0.000256  0.000256  0.004096   0.0   
1  0.013138       NaN        NaN      NaN  0.000256  0.000256  0.004096   0.0   
2  0.013267       NaN        NaN      NaN  0.000256  0.000256  0.004096   NaN   
3  0.015559       NaN        NaN      NaN  0.000256  0.000256  0.004096   NaN   
4  0.050943       NaN        NaN      NaN  0.000256  0.000256  0.004096   NaN   

   VelY  VelZ  
0   0.0   0.0  
1   0.0   0.0  
2   NaN   NaN  
3   NaN   NaN  
4   NaN   NaN  
time: 13.33319720000145


In [ ]:
# MERGE DATAFRAMES BACK TOGETHER





# TRIM DATA HEAD AND TAIL BY VELOCITY





In [ ]:
# FILTER 1:
# DELETE BAD DATA, THEN INTERPOLATE
'''
# Start performance counter
t1 = perf_counter()

print('\tdeleting bad values...\n')

# Interpolated 
ip_df = df.copy(deep=True)

# Loop through the dataframe, if standard  
# deviations are too high set that row to null
for i in range(0,len(df.GPS_Long)):
    if ip_df.SDn[i] > 0.005:
        ip_df.loc[i,:] = pd.NA
        
print(ip_df.info())

#End performance counter
t2 = perf_counter()
print(f'time: {t2-t1}')'''

In [ ]:
# INTERPOLATE GNSS DATA

t1 = perf_counter()

print(f'\tinterpolating "interpolated_dataframe" data...\n')
'''
# Interpolate all values in the df as floats
ip_df = ip_df.astype('float').interpolate()

# Convert time to int, status to string, 
ip_df['GPS_Status'] = ip_df['GPS_Status'].astype('string')

# Drop first row of values (uninterpolated null IMU values)
ip_df = ip_df.drop(index=0)
'''
print(f'\tinterpolating "standard dataframe" data...\n')

# Interpolate all values in the df as floats
df = df.astype('float').interpolate()

# Convert time to int, status to string, 
df['GPS_Status'] = df['GPS_Status'].astype('string')

# Drop first row of values (uninterpolated null IMU values)
df = df.drop(index=0)

print(df.info())

t2 = perf_counter()
print(f'time: {t2-t1}')


In [ ]:
# FILTER 2:
# COMPLEMENTARY FILTER

# How much time to get a valid reading (seconds)?
time_to_reading = 10

'''
** Need to know IMU orientation **
1. Get initial heading from 2 good GNSS points
2. Fuse accelerometer and gyro data to get accurate pitch, roll, yaw
3. Create a time-step gain from the pitch, roll, yaw and 

4. Set the optimized GNSS value to be:
    normalized_sd = (SDn - min(SDn))/max(SDn) - min(SDn)
    Lat = GNSS_Lat * (1-normalized_sd) + IMU * (normalized_sd)

5. Only overwrite GNSS values when error is high

'''

print(df.GPS_Long[2] - df.GPS_Long[1], 
                              df.GPS_Lat[2] - df.GPS_Lat[1])

RAD_TO_DEG = 180 / np.pi

i = 100

else:
    while df.SDn[i] > 0.0005:
        if df.SDn[i] < 0.0005:
            init_heading = np.arctan2(df.GPS_Long[i+1] - df.GPS_Long[i], 
                                      df.GPS_Lat[i+1] - df.GPS_Lat[i])
        else:
            i += 1

if init_heading < 0:
    init_heading += 2*np.pi
            
print(init_heading * RAD_TO_DEG)



In [ ]:
# PLOT UNFILTERED DATA
%matplotlib qt

print(df.info())

plt.title("Unfiltered GNSS Tracks")
plt.xlabel("longitude")
plt.ylabel("latitude")
#plt.scatter(ip_df.GPS_Long, ip_df.GPS_Lat)
            #cmap='copper_r')
plt.scatter(df.GPS_Long, df.GPS_Lat,
           c=df.SDn, cmap='copper_r')
plt.show()

In [ ]:
# WRITE DATA TO NEW CSVs

t1 = perf_counter()

# Write data
ip_df.to_csv('del_interp_data.csv')
df.to_csv('interp_data.csv')

t2 = perf_counter()
print(f'time: {t2-t1})


In [ ]:
# PLOT RAW IMU VALUES
%matplotlib qt

# Plot Linear Acceleration   
plt.figure(figsize=(20, 5))
plt.title('IMU Linear Acceleration')
plt.plot(df.Time, df.IMU_AngVelX,
        color='red')
plt.plot(df.Time, df.IMU_AngVelY,
        color='green')
plt.plot(df.Time, df.IMU_AngVelZ,
        color='blue')
plt.xlabel('IMU Time')
plt.ylabel('g (m/s^2)')
plt.show()

# Plot Angular Velocity
plt.figure(figsize=(20, 5))
plt.title('IMU Angular Velocity')
plt.plot(df.Time, df.IMU_LinearAccX,
        color='red')
plt.plot(df.Time, df.IMU_LinearAccY,
        color='green')
plt.plot(df.Time, df.IMU_LinearAccZ,
        color='blue')
plt.xlabel('IMU Time')
plt.ylabel('deg/sec')
plt.show()

In [ ]:
# RUN DATA THROUGH THE UNSCENTED KALMAN FILTER

'''
NOT YET WORKING
'''

# Import Extended Kalman Filter class from ekf_class.py
from ukf_filterpy import UKF

# Initialize the filter
ekf = EKF(init_x=0.0, 
          init_v=1.0, 
          init_z=0.0,
          accel_variance=0.1)

# Initialize new data frame to hold filtered values



# Loop through the data frame, updating the EKF at each step
for i in range(1, len(df)):
    
    # Set change in time via IMU time
    delta_t = df.Time[i]-df.Time[i-1]
    
    # Append mean and covariance EKF values to new data frame    
    filt_df.GNSS_lat.append(ekf.gnss_mean(0))
    filt_df.GNSS_lon.append(ekf.gnss_mean(1))
    filt_df.GNSS_alt.append(ekf.gnss_mean(2))
    
    filt_df.GNSS_lat_cov.append(ekf.gnss_cov(0))
    filt_df.GNSS_lon_cov.append(ekf.gnss_cov(1))
    filt_df.GNSS_alt_cov.append(ekf.gnss_cov(2))

    # Complete the EKF predict step (code in ekf_class.py)
    ekf.predict(dt=delta_t)
    
    # Complete the EKF update step (code in ekf_class.py)
    ekf.update( meas_value=,
                meas_variance=)



In [ ]:
# PLOT FILTERED AND UNFILTERED DATA

plt.title("Filtered vs Non-filtered")
plt.xlabel("longitude")
plt.ylabel("latitude")
plt.scatter(df.GPS_Long, df.GPS_Lat, 
            c='lightblue')
plt.scatter(filt_df.GNSS_lon, filt_df.GNSS_lat,
           c='red')
plt.show()
